<a href="https://colab.research.google.com/github/plaban1981/Huggingface_transformers_course/blob/main/Next_Word_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,Activation,Embedding
from string import punctuation
#
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from tensorflow.keras.utils import to_categorical

https://towardsdatascience.com/exploring-the-next-word-predictor-5e22aeb85d8f

https://renom.jp/notebooks/tutorial/time_series/text_generation_using_pretrained_word_embedding_layer/notebook.html

In [2]:
from google.colab import files
files.upload()

Saving HSBC_Credit_Card_FAQ.csv to HSBC_Credit_Card_FAQ.csv


{'HSBC_Credit_Card_FAQ.csv': b'Question,Answer\r\nWhich HSBC Credit Cards can I apply for?,"HSBC Credit Cards have been designed to complement your lifestyle. We currently offer credit cards which extend a host of offers ranging from travel privileges to dining, shopping and more.\nPlease visit the credit cards sectioncredit cards section View credit cards on offer from HSBC on our website for details on the various credit cards we offer, product features, benefits and the eligibility criteria."\r\nWhat documentation do I need to provide when applying for an HSBC Credit Card?,"Please note that the original documents need to be originally sighted and verified by a bank official:\n\x95\tSelf-attested recent passport size photograph of the applicant\n\x95\tSelf-attested copy of a valid photo identity document (please refer to the list below for details)\n\x95\tSelf-attested copy of your residence address proof (please refer to the list below for details)\n\x95\tSelf-attested copy of the l

In [3]:
import pandas as pd
df = pd.read_csv("/content/HSBC_Credit_Card_FAQ.csv",encoding='cp1252')

In [4]:
df.head()

,Question,Answer
0,Which HSBC Credit Cards can I apply for?,HSBC Credit Cards have been designed to comple...
1,What documentation do I need to provide when a...,Please note that the original documents need t...
2,How do I apply for an HSBC Credit Card?,To apply for an HSBC Credit Card you can:\n•\t...
3,"I have applied for an HSBC Credit Card, how ca...",You will receive timely alerts on the credit c...
4,How long does it take to receive an HSBC Credi...,Once we have received your application form co...


In [5]:
df.tail()

,Question,Answer
123,Can I decrease the credit limit on my Credit C...,"Yes, you may decrease the limit if your circum..."
124,Will I be intimated if my credit limit is redu...,"Yes, post the credit limit reduction you will ..."
125,Will my limit on my Add on card get increased/...,"Yes, if the limit on your primary credit card ..."
126,What happens if my credit limit request is dec...,If the credit request is rejected due to inter...
127,How do I contact HSBC for any queries or servi...,All account holders\nThe email address 'info@h...


In [6]:
text = df['Question'].values.tolist()

In [7]:
len(text)

128

In [8]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
words = []
for txt in text:
  words.extend(tokenizer.tokenize(txt.lower()))

In [9]:
words

['which',
 'hsbc',
 'credit',
 'cards',
 'can',
 'i',
 'apply',
 'for',
 'what',
 'documentation',
 'do',
 'i',
 'need',
 'to',
 'provide',
 'when',
 'applying',
 'for',
 'an',
 'hsbc',
 'credit',
 'card',
 'how',
 'do',
 'i',
 'apply',
 'for',
 'an',
 'hsbc',
 'credit',
 'card',
 'i',
 'have',
 'applied',
 'for',
 'an',
 'hsbc',
 'credit',
 'card',
 'how',
 'can',
 'i',
 'check',
 'on',
 'the',
 'status',
 'of',
 'this',
 'how',
 'long',
 'does',
 'it',
 'take',
 'to',
 'receive',
 'an',
 'hsbc',
 'credit',
 'card',
 'how',
 'can',
 'i',
 'check',
 'the',
 'status',
 'of',
 'my',
 'credit',
 'card',
 'consignment',
 'i',
 'have',
 'not',
 'received',
 'my',
 'card',
 'statement',
 'how',
 'do',
 'i',
 'register',
 'for',
 'e',
 'statements',
 'can',
 'i',
 'use',
 'my',
 'hsbc',
 'credit',
 'card',
 'as',
 'soon',
 'as',
 'i',
 'receive',
 'it',
 'how',
 'can',
 'i',
 'change',
 'my',
 'pin',
 'what',
 'fees',
 'and',
 'charges',
 'are',
 'linked',
 'to',
 'the',
 'credit',
 'card',
 

In [10]:
len(words)

1564

In [11]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))
#unique_word_index = dict((c, i) for i, c in enumerate(words))

In [ ]:
WORD_LENGTH = 3
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['which', 'hsbc', 'credit']
cards


In [ ]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [ ]:
print(np.argmax(X[0][0]))

279


## Building the model

In [ ]:
WORD_LENGTH

3

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

## Training

In [ ]:
from keras.optimizers import RMSprop
optimizer = RMSprop(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=500, shuffle=True).history

Epoch 1/500
12/12 [==============================] - 2s 62ms/step - loss: 5.6575 - accuracy: 0.0482 - val_loss: 5.6125 - val_accuracy: 0.1139
Epoch 2/500
12/12 [==============================] - 0s 5ms/step - loss: 5.5451 - accuracy: 0.1151 - val_loss: 5.3883 - val_accuracy: 0.0506
Epoch 3/500
12/12 [==============================] - 0s 5ms/step - loss: 5.1508 - accuracy: 0.0741 - val_loss: 4.9589 - val_accuracy: 0.0506
Epoch 4/500
12/12 [==============================] - 0s 5ms/step - loss: 4.7036 - accuracy: 0.0630 - val_loss: 4.9595 - val_accuracy: 0.0506
Epoch 5/500
12/12 [==============================] - 0s 5ms/step - loss: 4.5657 - accuracy: 0.0749 - val_loss: 5.0056 - val_accuracy: 0.0506
Epoch 6/500
12/12 [==============================] - 0s 5ms/step - loss: 4.5031 - accuracy: 0.0911 - val_loss: 5.0467 - val_accuracy: 0.0506
Epoch 7/500
12/12 [==============================] - 0s 5ms/step - loss: 4.4952 - accuracy: 0.0833 - val_loss: 5.0902 - val_accuracy: 0.0886
Epoch 8/500


In [ ]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("Which HSBC Credit ".lower())

which
hsbc
credit


array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [ ]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [ ]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    words = [unique_words[idx] for idx in next_indices]
    words = [ w for w in words if w not in text.split(" ")]
    return words

In [ ]:
import pickle
import heapq
q =  input()
print("correct sentence: ",q.lower())
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 3))

can i
correct sentence:  can i
Sequence:  can i
can
i
next possible words:  ['the', 'balance', 'my']


## New Method

## Tokenize input



In [12]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
words = []
for txt in text:
  words.extend(tokenizer.tokenize(txt.lower()))

In [13]:
len(words)

1564

## Preprocessing the data

* we are keeping training length to 3 so that we type atleast 3 words and have the next word predicted
* Generate Text Sequences
* Create features dictionary sequences. 
* Then encode it into the integer form with the help of the Tokenizer.

In [14]:
tokens = words
train_len = 3+1 # it tells how many words we need to type
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]


In [15]:
vocabulary_size

291

In [16]:
text_sequences

[['which', 'hsbc', 'credit', 'cards'],
 ['hsbc', 'credit', 'cards', 'can'],
 ['credit', 'cards', 'can', 'i'],
 ['cards', 'can', 'i', 'apply'],
 ['can', 'i', 'apply', 'for'],
 ['i', 'apply', 'for', 'what'],
 ['apply', 'for', 'what', 'documentation'],
 ['for', 'what', 'documentation', 'do'],
 ['what', 'documentation', 'do', 'i'],
 ['documentation', 'do', 'i', 'need'],
 ['do', 'i', 'need', 'to'],
 ['i', 'need', 'to', 'provide'],
 ['need', 'to', 'provide', 'when'],
 ['to', 'provide', 'when', 'applying'],
 ['provide', 'when', 'applying', 'for'],
 ['when', 'applying', 'for', 'an'],
 ['applying', 'for', 'an', 'hsbc'],
 ['for', 'an', 'hsbc', 'credit'],
 ['an', 'hsbc', 'credit', 'card'],
 ['hsbc', 'credit', 'card', 'how'],
 ['credit', 'card', 'how', 'do'],
 ['card', 'how', 'do', 'i'],
 ['how', 'do', 'i', 'apply'],
 ['do', 'i', 'apply', 'for'],
 ['i', 'apply', 'for', 'an'],
 ['apply', 'for', 'an', 'hsbc'],
 ['for', 'an', 'hsbc', 'credit'],
 ['an', 'hsbc', 'credit', 'card'],
 ['hsbc', 'credit', '

In [17]:
n_sequences.shape

(1560, 4)

In [ ]:
text_sequences[0]

['which', 'hsbc', 'credit', 'cards']

In [ ]:
n_sequences[0]

array([45, 13,  5, 57], dtype=int32)

In [30]:
text_sequences[1]

['hsbc', 'credit', 'cards', 'can']

In [31]:
n_sequences[1]

array([13,  5, 57, 10], dtype=int32)

* The numbers are nothing but the indexes of the respective words from the ‘sequences’ dictionary before re-assignment.

In [ ]:
vocabulary_size

291

* Once we have our sequences in encoded form training data and target data is defined by splitting the sequences into the inputs and output labels. 

* As for this example, we are going to predict the next word based on three previous words so in training we use the first three words as input and the last word as a label that is to be predicted by the model. Our ‘training_inputs’ would now be:

In [18]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape

(1560, 3)

In [19]:
seq_len

3

In [20]:
train_inputs[0],np.argmax(train_targets[0])

(array([45, 13,  5], dtype=int32), 57)

In [21]:
train_targets[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
np.argmax(train_targets[0])

57

In [22]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## GloVe Word Embeddings

* GloVe is an unsupervised learning algorithm to learn vector representation i.e word embedding for various words. 

* GloVe stands for Global Vectors for Word Representations. In this code, I will be using the 50-dimensional GloVe vectors for the task at hand.

## Define the embedding matrix 

* here all the words which are not in the GloVe dictionary being assigned a zero vector.

In [23]:
embeddings_index = dict()
f = open('/content/drive/MyDrive/NLP Tweets (1)/glove.6B.50d.txt','r')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [25]:
len(text_sequences)


1560

In [34]:
token_obj = {}
for i,j in zip(text_sequences, n_sequences):
  print(i,j)
  for cnt,item in enumerate(i):
    if item not in token_obj.keys():
      token_obj[item] = j[cnt]


['which', 'hsbc', 'credit', 'cards'] [45 13  5 57]
['hsbc', 'credit', 'cards', 'can'] [13  5 57 10]
['credit', 'cards', 'can', 'i'] [ 5 57 10  1]
['cards', 'can', 'i', 'apply'] [57 10  1 94]
['can', 'i', 'apply', 'for'] [10  1 94 11]
['i', 'apply', 'for', 'what'] [ 1 94 11  6]
['apply', 'for', 'what', 'documentation'] [ 94  11   6 150]
['for', 'what', 'documentation', 'do'] [ 11   6 150   8]
['what', 'documentation', 'do', 'i'] [  6 150   8   1]
['documentation', 'do', 'i', 'need'] [150   8   1  58]
['do', 'i', 'need', 'to'] [ 8  1 58 14]
['i', 'need', 'to', 'provide'] [  1  58  14 151]
['need', 'to', 'provide', 'when'] [ 58  14 151  76]
['to', 'provide', 'when', 'applying'] [ 14 151  76 152]
['provide', 'when', 'applying', 'for'] [151  76 152  11]
['when', 'applying', 'for', 'an'] [ 76 152  11  29]
['applying', 'for', 'an', 'hsbc'] [152  11  29  13]
['for', 'an', 'hsbc', 'credit'] [11 29 13  5]
['an', 'hsbc', 'credit', 'card'] [29 13  5  3]
['hsbc', 'credit', 'card', 'how'] [13  5  3 

In [35]:
token_obj

{'a': 26,
 'about': 186,
 'accepted': 267,
 'account': 40,
 'across': 201,
 'activity': 193,
 'add': 144,
 'additional': 98,
 'affect': 277,
 'affected': 199,
 'after': 146,
 'all': 268,
 'allocation': 147,
 'along': 289,
 'am': 50,
 'amount': 43,
 'an': 29,
 'and': 38,
 'another': 225,
 'any': 82,
 'anything': 173,
 'applied': 113,
 'apply': 94,
 'applying': 152,
 'are': 80,
 'as': 78,
 'ask': 259,
 'assurance': 250,
 'at': 105,
 'atm': 128,
 'authentication': 54,
 'automatically': 185,
 'avail': 69,
 'available': 166,
 'availed': 73,
 'balance': 25,
 'bank': 67,
 'banking': 103,
 'be': 15,
 'been': 191,
 'being': 87,
 'benefit': 89,
 'blocked': 216,
 'by': 111,
 'calculated': 172,
 'can': 10,
 'card': 3,
 'cards': 57,
 'case': 109,
 'cash': 33,
 'change': 97,
 'changed': 271,
 'charged': 39,
 'charges': 120,
 'check': 114,
 'cheque': 178,
 'chip': 44,
 'choice': 238,
 'claim': 196,
 'clear': 179,
 'close': 34,
 'coming': 272,
 'commerce': 248,
 'complete': 207,
 'consignment': 153,
 

In [51]:
len(token_obj)


290

* Here we have defined maxLen which is the maximum length of one review we will use for our training as 50

In [41]:
embedding_matrix = np.zeros((vocabulary_size, 50))
#token_obj = dict(zip(text_sequences, n_sequences))
for word, i in token_obj.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [37]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.11891   ,  0.15255   , -0.082073  , ..., -0.57511997,
        -0.26671001,  0.92120999],
       [ 0.41800001,  0.24968   , -0.41242   , ..., -0.18411   ,
        -0.11514   , -0.78580999],
       ...,
       [-0.45288   , -0.40994999,  0.4156    , ..., -0.37878999,
         0.27081999,  0.032871  ],
       [ 0.53207999,  0.39724001,  0.26486999, ..., -1.21130002,
        -0.36894   , -0.74188   ],
       [ 0.057915  , -0.42254999,  0.51775002, ...,  0.79315001,
        -0.050581  ,  0.78106999]])

In [38]:
embedding_matrix.shape

(291, 50)

## Build Model

## Define the embedding layer using the built-in Keras Embedding layer.

* The embedding layer maps the words to their embedding vectors from the embedding matrix. 
* We can keep this layer as trainable, which will learn the word embedding itself but as we are using GloVe Embeddings, we won’t keep the layer as trainable.

In [43]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


model = Sequential()
model.add(Embedding(vocabulary_size, 50, input_length=50,weights = [embedding_matrix], trainable=False))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,validation_split=0.10,epochs=600,verbose=1)
model.save("mymodel.h5")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            14550     
_________________________________________________________________
lstm (LSTM)                  (None, 50, 50)            20200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 291)               14841     
Total params: 72,341
Trainable params: 57,791
Non-trainable params: 14,550
_________________________________________________________________
None
Epoch 1/600
44/44 [==============================] - 7s 23ms/step - loss: 5.5063 - accuracy: 0.0178 - val_

## Copy the saved model to google drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import shutil
shutil.copy("/content/mymodel.h5","/content/drive/MyDrive/ZeonAI/")

'/content/drive/MyDrive/ZeonAI/mymodel.h5'

## Make Inference on new text for the next 3 possible words

In [50]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

Which SBI credit
[45, 5] [[ 0 45  5]]
Next word suggestion: cards
Next word suggestion: period
Next word suggestion: can


In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from tensorflow.keras.utils import to_categorical
#
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
#
tokenizer = Tokenizer()
seq_len =3
def next_word_model():
  model = Sequential()
  model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
  model.add(LSTM(50,return_sequences=True))
  model.add(LSTM(50))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(vocabulary_size, activation='softmax'))
  print(model.summary())
  return model
  


## Loading the saved model and making inference

In [ ]:
from keras.preprocessing.sequence import pad_sequences
"""
model = next_word_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights("/content/mymodel.h5")
#model = load_model("/content/drive/MyDrive/ZeonAI/mymodel.h5")
"""
model = next_word_model()
model = load_model("/content/drive/MyDrive/ZeonAI/mymodel.h5")
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
print(encoded_text,seq_len)
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 3, 3)              873       
_________________________________________________________________
lstm_16 (LSTM)               (None, 3, 50)             10800     
_________________________________________________________________
lstm_17 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_16 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_17 (Dense)             (None, 291)               14841     
Total params: 49,264
Trainable params: 49,264
Non-trainable params: 0
_________________________________________________________________
None
HSBC for any 
[13, 11, 82] 3
[[13 11 82]]
Next word suggestion: queries
Next word suggestion: as
Next word sugg